In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('fact_orders.csv')
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,days_to_delivery,estimated_days_to_delivery,delay_flag_estimated,delay_flag_maen
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,15,0,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,12,17,0,1
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,26,0,0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,26,0,1
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,12,0,0


### dim_date 만들기

In [2]:
from pyspark.sql import SparkSession
# Spark 세션 생성
spark = SparkSession.builder.appName("olist").getOrCreate()

spk_orders = spark.read.csv('fact_orders.csv', header=True, inferSchema=True)
spk_orders.createOrReplaceTempView("orders")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/01 09:42:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, to_date, date_format
import pandas as pd

# Spark 세션 생성
spark = SparkSession.builder.appName("CreateDimDate").getOrCreate()

# ✅ 1. 날짜 범위 생성 (2016년 1월 1일 ~ 2019년 1월 1일)
date_range = pd.date_range(start="2016-01-01", end="2019-01-01", freq='D')
date_df = pd.DataFrame({'full_date': date_range})  # Pandas DataFrame 생성
date_df

25/03/01 06:29:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


,full_date
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
1092,2018-12-28
1093,2018-12-29
1094,2018-12-30
1095,2018-12-31


- withColumn 함수를 사용하여 매핑된 칼럼을 생성

In [14]:
from pyspark.sql.functions import col, to_date, expr, date_format, from_utc_timestamp


dim_date = spark.createDataFrame(date_df)

# 날짜 변환: 브라질 시간대 설정 (UTC → UTC-3 변환)
dim_date = dim_date.withColumn("full_date", to_date(from_utc_timestamp(col("full_date"), "America/Sao_Paulo")))


dim_date = dim_date.withColumn("year", expr("YEAR(full_date)"))
dim_date = dim_date.withColumn("month", expr("MONTH(full_date)"))
dim_date = dim_date.withColumn("day", expr("DAY(full_date)"))
dim_date = dim_date.withColumn("week", expr("WEEKOFYEAR(full_date)"))
dim_date = dim_date.withColumn("quarter", expr("QUARTER(full_date)"))


dim_date = dim_date.withColumn("day_of_week", date_format(col("full_date"), "EEEE"))  # 'Monday', 'Tuesday', ...
dim_date = dim_date.withColumn("is_weekend", expr("CASE WHEN day_of_week IN ('Saturday', 'Sunday') THEN 1 ELSE 0 END"))

dim_date.show(10)



+----------+----+-----+---+----+-------+-----------+----------+
| full_date|year|month|day|week|quarter|day_of_week|is_weekend|
+----------+----+-----+---+----+-------+-----------+----------+
|2015-12-31|2015|   12| 31|  53|      4|   Thursday|         0|
|2016-01-01|2016|    1|  1|  53|      1|     Friday|         0|
|2016-01-02|2016|    1|  2|  53|      1|   Saturday|         1|
|2016-01-03|2016|    1|  3|  53|      1|     Sunday|         1|
|2016-01-04|2016|    1|  4|   1|      1|     Monday|         0|
|2016-01-05|2016|    1|  5|   1|      1|    Tuesday|         0|
|2016-01-06|2016|    1|  6|   1|      1|  Wednesday|         0|
|2016-01-07|2016|    1|  7|   1|      1|   Thursday|         0|
|2016-01-08|2016|    1|  8|   1|      1|     Friday|         0|
|2016-01-09|2016|    1|  9|   1|      1|   Saturday|         1|
+----------+----+-----+---+----+-------+-----------+----------+
only showing top 10 rows



In [ ]:
# dim_date.toPandas().to_csv('dim_date.csv', index=False)

In [4]:
dim_date = spark.read.csv('dim_date.csv', header=True, inferSchema=True)
dim_date.createOrReplaceTempView("dim_date")

In [5]:
fact_orders_date = spk_orders.join(dim_date,on=spk_orders.order_purchase_timestamp.cast("date") == dim_date.full_date,how='left')
fact_orders_date.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------+--------------------------+--------------------+---------------+----------+----+-----+---+----+-------+-----------+----------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|days_to_delivery|estimated_days_to_delivery|delay_flag_estimated|delay_flag_maen| full_date|year|month|day|week|quarter|day_of_week|is_weekend|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------+--------------------------+--------------------+---------------+----------+----+-----+---+----+-------+-----------+----------

In [7]:
fact_orders_date.createOrReplaceTempView("fact_orders_date")

- 월 별 days_to_delivery 구하기

In [10]:
query = """
SELECT AVG(days_to_delivery) AS avg_days_to_delivery,
        month
  FROM fact_orders_date
 GROUP BY  month
 ORDER BY  month
"""
spark.sql(query).show(10)

+--------------------+-----+
|avg_days_to_delivery|month|
+--------------------+-----+
|  11.071378091872791|    1|
|  12.389874810840556|    2|
|  11.517286135693215|    3|
|    9.94654015274242|    4|
|   9.619757118073272|    5|
|   8.587733272644515|    6|
|   8.452512254901961|    7|
|   7.928440543929019|    8|
|  10.154037886340976|    9|
|  10.154861111111112|   10|
+--------------------+-----+
only showing top 10 rows



In [12]:
query = """
SELECT AVG(days_to_delivery) AS avg_days_to_delivery,
        MONTH(order_purchase_timestamp) AS month
  FROM fact_orders_date
 GROUP BY  MONTH(order_purchase_timestamp) 
 ORDER BY  MONTH(order_purchase_timestamp)

"""
spark.sql(query).show(10)

+--------------------+-----+
|avg_days_to_delivery|month|
+--------------------+-----+
|  11.071378091872791|    1|
|  12.389874810840556|    2|
|  11.517286135693215|    3|
|    9.94654015274242|    4|
|   9.619757118073272|    5|
|   8.587733272644515|    6|
|   8.452512254901961|    7|
|   7.928440543929019|    8|
|  10.154037886340976|    9|
|  10.154861111111112|   10|
+--------------------+-----+
only showing top 10 rows

